In [11]:
import csv
import pandas as pd
import hashlib
from datetime import datetime


In [12]:
import getpass

In [13]:
def hash_password(password):
    # Hash the password using SHA-256
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    return hashed_password

In [14]:
def valid_date(birth_date):
    try:
        datetime.strptime(birth_date,"%Y-%m-%d")
        return True
    except ValueError: 
        return False
        

In [15]:
def signup():
    fields=["Name","Column","Birthdate"]
    name = input("Enter your name: ")
    password = getpass.getpass("Enter your password: ")
    hashed_password=hash_password(password)
    
    
    birthdate = input("Enter your birthdate (YYYY-MM-DD): ")
    while not valid_date(birthdate):
        print("Invalid date")
        birthdate=input("Enter your birthdate (YYYY-MM-DD):")



    with open('user.csv', 'a', newline='') as scorefile:
        fieldnames=['Name','Password','Birthdate']
        score_writer = csv.DictWriter(scorefile, fieldnames=fieldnames)

        if scorefile.tell()==0:
            score_writer.writeheader()

        score_writer.writerow({'Name':name, 'Password':hashed_password,'Birthdate':birthdate})
    print("Signup successful!")
    display_questions(name)
    



In [16]:
def login():
    name = input("Enter your name: ").strip().lower()
    password = getpass.getpass("Enter your password: ")

    df = pd.read_csv('user.csv')

    df.columns = df.columns.str.lower()
    df['name'] = df['name'].str.lower()

    if 'name' not in df.columns or 'password' not in df.columns:
        print("User not found. Please signup.")
        return


    name_index = df.columns.get_loc('name')
    password_index = df.columns.get_loc('password')

    for index, row in df.iterrows():
        if row.iloc[name_index].strip() == name:
            if row.iloc[password_index] == hash_password(password):          
                print("Login successful!")
                display_questions(name)
                return
            else:
                print("Invalid password!")
                return

    print("User not found. Please signup.")


In [17]:
def create_questions_csv():
    questions = [
        (1, "What is the capital of France?"),
        (2, "Who wrote 'Harry Potter'?"),
        (3, "What is the chemical symbol for water?")
    ]
    
    with open('questions.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Question"])  # Write column headers
        for question_id, question_text in questions:
            writer.writerow([question_id, question_text])


def create_answers_csv():
    answers = [
        (1, "Paris", "London", "Berlin", "Rome", "A"),
        (2, "William Shakespeare", "Jane Austen", "JK Rowling", "Toni Morrison", "C"),
        (3, "CO2", "SO4", "H2O", "H2O2", "C")
    ]
    
    with open('answers.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Option A", "Option B", "Option C", "Option D", "Correct Answer"])  # Write column headers
        for answer in answers:
            writer.writerow(answer)

In [18]:
def display_questions(name):
    questions_df = pd.read_csv('questions.csv')
    answers_df = pd.read_csv('answers.csv')

    total_score = 0
    for index, row in questions_df.iterrows():
        question = row['Question']

        # Accessing options and correct answer from answers_df
        options = [answers_df.iloc[index]['Option A'], 
           answers_df.iloc[index]['Option B'], 
           answers_df.iloc[index]['Option C'], 
           answers_df.iloc[index]['Option D']]
        correct_answer = answers_df.iloc[index]['Correct Answer']

        print(question)
        print(f"A. {options[0]:<20} B. {options[1]}")
        print(f"C. {options[2]:<20} D. {options[3]}")

        user_answer = input("Your answer: ").upper()
        print("\n")
        if user_answer == correct_answer:
            total_score += 1

    print(f"Total Score: {total_score}/{len(questions_df)}")

    with open('user_scores.csv', 'a', newline='') as scorefile:
        fieldnames = ['name', 'score']
        score_writer = csv.DictWriter(scorefile, fieldnames=fieldnames)

        # If file is empty, write header
        if scorefile.tell() == 0:
            score_writer.writeheader()

        score_writer.writerow({'name': name, 'score': total_score})


In [19]:
def main():
    create_questions_csv()
    create_answers_csv()

    print("----------Welcome---------")
    while True:
        print("\n1. Signup")
        print("2. Login")
        print("3. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            signup()
        elif choice == '2':
            login()


        elif choice == '3':
            break
        else:
            print("Invalid choice. Please try again.")

In [20]:
if __name__ == "__main__":
    main()

----------Welcome---------

1. Signup
2. Login
3. Exit
Enter your choice: 2
Enter your name: bharti
Enter your password: ········
Login successful!
What is the capital of France?
A. Paris                B. London
C. Berlin               D. Rome
Your answer: a


Who wrote 'Harry Potter'?
A. William Shakespeare  B. Jane Austen
C. JK Rowling           D. Toni Morrison
Your answer: c


What is the chemical symbol for water?
A. CO2                  B. SO4
C. H2O                  D. H2O2
Your answer: c


Total Score: 3/3

1. Signup
2. Login
3. Exit
Enter your choice: 3
